# Hands On Tutorial 3: Multitask Deep Learning for Drug Discovery

Implemntation of the model described in https://arxiv.org/pdf/1502.02072.pdf

### Imports and parameters

In [1]:
import keras
import keras.backend as K
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.optimizers import SGD

import pandas as pd
import numpy as np
import zipfile
import requests
import shutil
import os

from helpers.extra_metrics import explained_variance_score, r2_score

Using TensorFlow backend.


In [2]:
""" 
add imports for layers and loss functions here
"""
from keras.layers import Input, Dense, Conv1D, LSTM, Dropout, Flatten
from keras.losses import MSE

In [3]:
MAX_LEN = 120
VAL_SIZE = 20000
LEARNING_RATE = .001
EPOCHS = 50
BATCH_SIZE = 64

### Dataset preparation

We will use the [QM9 dataset](http://quantum-machine.org/datasets/):
* L. Ruddigkeit, R. van Deursen, L. C. Blum, J.-L. Reymond, Enumeration of 166 billion organic small molecules in the chemical universe database GDB-17, J. Chem. Inf. Model. 52, 2864–2875, 2012.
* R. Ramakrishnan, P. O. Dral, M. Rupp, O. A. von Lilienfeld, Quantum chemistry structures and properties of 134 kilo molecules, Scientific Data 1, 140022, 2014. [bibtex]

The QM9 dataset contains 133885 organic molecules, represented as SMILES strings, with 15 properties each. We will train a regression model to predict all 15 properties in a multitask settings.

In [3]:
# download dataset
url = 'https://s3.us-east-2.amazonaws.com/weizmann-dl-workshop/data_qm9.pkl.zip'
file_path = 'datasets/qm9.pkl.zip'

In [4]:
os.makedirs('datasets/')
r = requests.get(url, auth=('usrname', 'password'), verify=False,stream=True)
r.raw.decode_content = True
with open(file_path, 'wb') as f:
        shutil.copyfileobj(r.raw, f)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [5]:
""" ONLY IF THE PREVIOUS FAILED TO DOWNLOAD """
!mkdir -p datasets
!wget --no-check-certificate $url -O $file_path

--2018-03-13 11:37:41--  https://s3.us-east-2.amazonaws.com/weizmann-dl-workshop/data_qm9.pkl.zip
Resolving s3.us-east-2.amazonaws.com... 52.219.88.91
Connecting to s3.us-east-2.amazonaws.com|52.219.88.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104327424 (99M) [application/zip]
Saving to: ‘datasets/qm9.pkl.zip’

datasets/qm9.pkl.zi 100%[===================>]  99.49M   327KB/s    in 3m 37s  

2018-03-13 11:41:19 (470 KB/s) - ‘datasets/qm9.pkl.zip’ saved [104327424/104327424]



In [6]:
# extract dataset
with zipfile.ZipFile(file_path, 'r') as file:
    file.extractall('datasets/')

In [4]:
# preprocess data
data = pd.read_pickle('datasets/data_qm9.pkl')
labels = ['A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'U0', 'U', 'H', 'G', 'Cv']

properties = {}
for i, key in enumerate(labels):
    properties[key] = data['properties'][:, i]
    
properties['smiles'] = data['smiles_optimized']

df = pd.DataFrame(properties)
df = df[['smiles'] + labels]
df.head()

,smiles,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv
0,C,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.478930,-40.476062,-40.475117,-40.498597,6.469
1,N,293.60975,293.541110,191.393970,1.6256,9.46,-0.2570,0.0829,0.3399,26.1563,0.034358,-56.525887,-56.523026,-56.522082,-56.544961,6.316
2,O,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,0.3615,19.0002,0.021375,-76.404702,-76.401867,-76.400922,-76.422349,6.002
3,C#C,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574
4,C#N,0.00000,44.593883,44.593883,2.8937,12.99,-0.3604,0.0191,0.3796,48.7476,0.016601,-93.411888,-93.409370,-93.408425,-93.431246,6.278


In [5]:
# set maximum length
MAX_LEN = df.smiles.str.len().max()

#### Task 1: convert smiles strings into one-hot representation and add to the dataframe in a column named 'one_hot' 
  
hint: use zero padding to make the length equal to MAX_LEN

In [6]:
"""
Convert SMILES to one-hot here
"""

smiles = df.smiles.tolist()
characters = set()

for s in smiles:
    for char in s:
        if char not in characters:
            characters.add(char)

characters = sorted(list(characters))
num_chars = len(characters)

print('Number of strings:', len(smiles))
print('Number of unique chars:', num_chars)

chars_index = dict([(char, i) for i, char in enumerate(characters)])

one_hot = np.zeros((len(df), MAX_LEN, num_chars), dtype='float32')

for i, s in enumerate(smiles):
    for j, char in enumerate(s):
        one_hot[i, j, chars_index[char]] = 1.
        
df['one_hot'] = one_hot.tolist()
df.head()

Number of strings: 133885
Number of unique chars: 21


,smiles,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,U0,U,H,G,Cv,one_hot
0,C,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.478930,-40.476062,-40.475117,-40.498597,6.469,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,N,293.60975,293.541110,191.393970,1.6256,9.46,-0.2570,0.0829,0.3399,26.1563,0.034358,-56.525887,-56.523026,-56.522082,-56.544961,6.316,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,O,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,0.3615,19.0002,0.021375,-76.404702,-76.401867,-76.400922,-76.422349,6.002,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,C#C,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,C#N,0.00000,44.593883,44.593883,2.8937,12.99,-0.3604,0.0191,0.3796,48.7476,0.016601,-93.411888,-93.409370,-93.408425,-93.431246,6.278,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [25]:
# split to train/validation sets
x_train = np.array([np.array(oh) for oh in df.one_hot.values[:-VAL_SIZE]])
x_val = np.array([np.array(oh) for oh in df.one_hot.values[-VAL_SIZE:]])

y_train = df[labels].values[:-VAL_SIZE]
y_val = df[labels].values[-VAL_SIZE:]

### Building Model Architecture

#### Task 2: define model architecture

In [8]:
inp = Input(shape=(MAX_LEN, num_chars))

In [9]:
"""
Build your model here.
Eventually the variable 'outputs' should be a list of all the model outputs, one for each task.
"""
final_hidden_layer = Flatten()(inp)

In [10]:
output = Dense(units=15, 
               activation='linear', 
               kernel_regularizer=None)(final_hidden_layer)

In [11]:
model = Model(inputs=inp, outputs=output)

### Compile model and set loss function and metrics


#### Task 3: define loss function

In [12]:
"""
Define your loss function here
"""
loss = MSE

In [13]:
# define metrics
metrics = [MSE, explained_variance_score, r2_score]

# compile model
model.compile(loss=loss,
              metrics=metrics,
              optimizer=SGD(lr=LEARNING_RATE, nesterov=True))

### Model Training


In [14]:
# set callbacks
callbacks = [ReduceLROnPlateau(factor=.2, patience=4, verbose=1),
             EarlyStopping(monitor='val_loss', patience=10)]

In [ ]:
# train model
model.fit(x=x_train,
          y=y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          callbacks=callbacks,
          validation_data=(x_val, y_val))

Train on 113885 samples, validate on 20000 samples
Epoch 1/50
 53248/113885 [=============>................] - ETA: 2s - loss: 552537.2646 - mean_squared_error: 552537.2646 - explained_variance_score: -14.8484 - r2_score: 0.9686

### Model Evaluation


In [37]:
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score

In [28]:
y_pred = model.predict(x=x_val)

In [38]:
print('##### Model Evaluation #####')
print('Mean Squared Error: %f' % mean_squared_error(y_true=y_val, y_pred=y_pred))
print('Explained Variance: %f' % explained_variance_score(y_true=y_val, y_pred=y_pred))
print('R2 Score: %f' % r2_score(y_true=y_val, y_pred=y_pred))

##### Model Evaluation #####
Mean Squared Error: 36600.802164
Explained Variance: -6.136137
R2 Score: -3.598258


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/regression.py:482: DeprecationWarning: Default 'multioutput' behavior now corresponds to 'variance_weighted' value which is deprecated since 0.17, it will be changed to 'uniform_average' starting from 0.19.
  DeprecationWarning)
